In [27]:
import pandas as pd
import os
import re
import string
import matplotlib.pyplot as plt
import nltk
import numpy as np
import seaborn as sns


from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

In [28]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import confusion_matrix, classification_report

In [29]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [30]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
# Data processing
import pandas as pd

# Modeling
from transformers import pipeline
classifier = pipeline(task="zero-shot-classification", 
                      model="facebook/bart-large-mnli",
                      device=0) 

In [32]:
data=pd.read_csv('query_dataset.csv')
data.head(10)

,Main_Label,Sub_topic,Material,Description,Keywords,0,0.1,Unnamed: 7,0.2,0.3,...,0.16,0.17,0.18,0.19,0.20,0.21,0.22,0.23,0.24,text
0,Data Structures and Algorithms,Arrays,"Introduction to arrays, declaration, and manip...",Arrays are one of the most fundamental data st...,declaration of arrays,accessing elements of arrays,"and manipulating arrays.""","Data structures, arrays, declaration, manipula...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Data Structures and AlgorithmsArraysIntroducti...
1,Data Structures and Algorithms,Arrays,Multi-dimensional arrays,A multi-dimensional array is an array of array...,Data structures,arrays,multi-dimensional arrays,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Data Structures and AlgorithmsArraysMulti-dime...
2,Data Structures and Algorithms,Arrays,Sorting and searching arrays,Sorting and searching algorithms are some of t...,including linear search,binary search,bubble sort,selection sort,insertion sort,and quicksort.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Data Structures and AlgorithmsArraysSorting an...
3,Data Structures and Algorithms,Sorting,Bubble sort,Bubble sort is a simple sorting algorithm that...,compares each pair of adjacent items,and swaps them if they are in the wrong order...,Sorting,algorithms,bubble sort,time complexity,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Data Structures and AlgorithmsSortingBubble so...
4,Data Structures and Algorithms,Sorting,Selection sort,Selection sort is a simple sorting algorithm t...,Sorting,algorithms,selection sort,time complexity,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Data Structures and AlgorithmsSortingSelection...
5,Data Structures and Algorithms,Sorting,Insertion sort,Insertion sort is a simple sorting algorithm t...,Sorting,algorithms,insertion sort,time complexity,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Data Structures and AlgorithmsSortingInsertion...
6,Data Structures and Algorithms,Sorting,Quick sort,Quick sort is a divide-and-conquer algorithm t...,then recursively sorts the subarrays. This ma...,Sorting,algorithms,quick sort,time complexity,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Data Structures and AlgorithmsSortingQuick sor...
7,Data Structures and Algorithms,Sorting,Merge sort,Merge sort is a divide-and-conquer algorithm t...,recursively sorts each half,and then merges the sorted halves. This mater...,Sorting,algorithms,merge sort,time complexity,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Data Structures and AlgorithmsSortingMerge sor...
8,Data Structures and Algorithms,Stacks and Queues,Introduction to stacks,A stack is a data structure that stores a coll...,with operations to push and pop elements. Thi...,stack operations,and applications of stacks.,Data structures,stacks,push,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Data Structures and AlgorithmsStacks and Queue...
9,Data Structures and Algorithms,Stacks and Queues,Introduction to queues,A queue is a data structure that stores a coll...,with operations to enqueue (add) elements to ...,queue operations,and applications of queues.,Data structures,queues,enqueue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Data Structures and AlgorithmsStacks and Queue...


In [33]:
df = data[['Main_Label', 'Sub_topic','text']].copy()
df.head()

,Main_Label,Sub_topic,text
0,Data Structures and Algorithms,Arrays,Data Structures and AlgorithmsArraysIntroducti...
1,Data Structures and Algorithms,Arrays,Data Structures and AlgorithmsArraysMulti-dime...
2,Data Structures and Algorithms,Arrays,Data Structures and AlgorithmsArraysSorting an...
3,Data Structures and Algorithms,Sorting,Data Structures and AlgorithmsSortingBubble so...
4,Data Structures and Algorithms,Sorting,Data Structures and AlgorithmsSortingSelection...


In [34]:
df.describe()

,Main_Label,Sub_topic,text
count,303,303,297
unique,26,148,297
top,Probabilistic and Statistical Mathematics,Introduction,Data Structures and AlgorithmsArraysIntroducti...
freq,21,7,1


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Main_Label  303 non-null    object
 1   Sub_topic   303 non-null    object
 2   text        297 non-null    object
dtypes: object(3)
memory usage: 7.6+ KB


In [36]:
df = df.dropna()

In [37]:
df.reset_index(inplace = True)

In [38]:
def replace_abbrev(text):
    text = re.sub(r"what's", "what is ",text)    
    text = re.sub(r"\'ve", " have ",text)
    text = re.sub(r"(\w+)(n't)", r"\1 not ",text)
    text = re.sub(r"i\'m", "i am ",text)
    text = re.sub(r"\'re", " are ",text)
    text = re.sub(r"\'d", " would ",text)
    text = re.sub(r"\'ll", " will ",text)
    text = re.sub(r"\'scuse", "excuse",text)
    text = re.sub(r"\'s", " ",text)
     # complete -ing
    text = re.sub(r'(\w+in)(\')(\s)', r'\1g\3', text)
    text = text.strip()
    text=text.lower()
    return text


def replace_url(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    text = text.strip()
    return text


def remove_punctuations(words):
    for word in words:
        if word in string.punctuation:
            words = words.replace(word, '')
    return words

def replace_emoji(data):
  emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
  data = emoji_pattern.sub(r'', data)
  data = data.strip()
  return(data)


def Remove_stopwords(data):
  stop = stopwords.words('english')
  data = " ".join([word for word in data.split() if word not in (stop)])
  data = data.strip()
  return data


def remove_extras(data):
  data = re.sub('\s', ' ', data)
  data = re.sub('\n', ' ', data)
  # data = re.sub(r"[^a-z.!?\']", " ", data)
  # remove extra spaces
  data = data.strip()
  return data


In [39]:
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize_word(data):
  return WordNetLemmatizer().lemmatize(data) 

In [40]:
def clean_data(data):
  #replace abbre
  # data=data.apply(replace_abbrev)
  # #remove stopwords
  data=data.apply(Remove_stopwords)
  # #remove punct
  data=data.apply(remove_punctuations)
  # #replace emoji
  data = data.apply(replace_emoji)
  #remove extras
  data = data.apply(remove_extras)
  # #lemmitization
  data=data.apply(lemmatize_word)
  return data

In [41]:
x_train=df["text"]

In [42]:
x_train[:100]

0     Data Structures and AlgorithmsArraysIntroducti...
1     Data Structures and AlgorithmsArraysMulti-dime...
2     Data Structures and AlgorithmsArraysSorting an...
3     Data Structures and AlgorithmsSortingBubble so...
4     Data Structures and AlgorithmsSortingSelection...
                            ...                        
95    Theory of ComputationFinite AutomataSlidesThis...
96    Theory of ComputationRegular LanguagesBookIntr...
97    Theory of ComputationContext-Free GrammarsVide...
98    Theory of ComputationPumping LemmaArticleAn ar...
99    Theory of ComputationDecidabilitySlidesThis se...
Name: text, Length: 100, dtype: object

In [43]:
train_data=clean_data(x_train)

In [44]:
df["clean_data"]=train_data

In [45]:
df.head()

,index,Main_Label,Sub_topic,text,clean_data
0,0,Data Structures and Algorithms,Arrays,Data Structures and AlgorithmsArraysIntroducti...,Data Structures AlgorithmsArraysIntroduction a...
1,1,Data Structures and Algorithms,Arrays,Data Structures and AlgorithmsArraysMulti-dime...,Data Structures AlgorithmsArraysMultidimension...
2,2,Data Structures and Algorithms,Arrays,Data Structures and AlgorithmsArraysSorting an...,Data Structures AlgorithmsArraysSorting search...
3,3,Data Structures and Algorithms,Sorting,Data Structures and AlgorithmsSortingBubble so...,Data Structures AlgorithmsSortingBubble sortBu...
4,4,Data Structures and Algorithms,Sorting,Data Structures and AlgorithmsSortingSelection...,Data Structures AlgorithmsSortingSelection sor...


In [46]:
df.Main_Label.unique()

array(['Data Structures and Algorithms',
       'Design and Analysis of Algorithms', 'Operating Systems',
       'Computer Networks', 'Computer System Architecture',
       'Database Management Systems', 'Object Oriented Programming',
       'Software Engineering', 'Theory of Computation', 'Deep Learning',
       'Machine Learning', 'Natural Language Processing',
       'Artificial Intelligence', 'Automata Theory', 'Compiler Design',
       'Cybersecurity', 'Discrete Mathematics', 'Calculus Maths',
       'Linear Algebra', 'Probabilistic and Statistical Mathematics',
       'Engineering Physics', 'Engineering Chemistry',
       'Mechanical Engineering', 'Civil Engineering', 'Basic Electronics'],
      dtype=object)

In [47]:
df.Sub_topic.unique()

array(['Arrays', 'Sorting', 'Stacks and Queues', 'Linked Lists',
       'Introduction', 'Divide and Conquer', 'Greedy Algorithms',
       'Dynamic Programming', 'Backtracking', 'Processes', 'Threads',
       'Memory Management', 'File Systems', 'Security', 'Networking',
       'Virtualization', 'Introduction to Computer Networks',
       'Network Types', 'Physical Layer', 'Data Link Layer',
       'Network Layer', 'Transport Layer', 'Application Layer',
       'CPU Architecture', 'Memory Hierarchy', 'Input/Output Devices',
       'Instruction Set Architecture (ISA)',
       'Relational Database Management Systems (RDBMS)',
       'NoSQL Databases', 'Database Architecture', 'Database Design',
       'Object-Oriented Concepts', 'Java Programming', 'C++ Programming',
       'Python Programming', 'Software Development Life Cycle',
       'Agile Development', 'Requirements Engineering', 'Software Design',
       'Software Testing', 'Software Maintenance',
       'Software Configuration Mana

In [48]:
# Put reviews in a list
sequences = df['clean_data'].to_list()

# Define the candidate labels 
candidate_labels = ['Data Structures and Algorithms',
       'Design and Analysis of Algorithms', 'Operating Systems',
       'Computer Networks', 'Computer System Architecture',
       'Database Management Systems', 'Object Oriented Programming',
       'Software Engineering', 'Theory of Computation', 'Deep Learning',
       'Machine Learning', 'Natural Language Processing',
       'Artificial Intelligence', 'Automata Theory', 'Compiler Design',
       'Cybersecurity', 'Discrete Mathematics', 'Calculus Maths',
       'Linear Algebra', 'Probabilistic and Statistical Mathematics',
       'Engineering Physics', 'Engineering Chemistry',
       'Mechanical Engineering', 'Civil Engineering', 'Basic Electronics','Arrays', 'Sorting', 'Stacks and Queues', 'Linked Lists',
       'Introduction', 'Divide and Conquer', 'Greedy Algorithms',
       'Dynamic Programming', 'Backtracking', 'Processes', 'Threads',
       'Memory Management', 'File Systems', 'Security', 'Networking',
       'Virtualization', 'Introduction to Computer Networks',
       'Network Types', 'Physical Layer', 'Data Link Layer',
       'Network Layer', 'Transport Layer', 'Application Layer',
       'CPU Architecture', 'Memory Hierarchy', 'Input/Output Devices',
       'Instruction Set Architecture (ISA)',
       'Relational Database Management Systems (RDBMS)',
       'NoSQL Databases', 'Database Architecture', 'Database Design',
       'Object-Oriented Concepts', 'Java Programming', 'C++ Programming',
       'Python Programming', 'Software Development Life Cycle',
       'Agile Development', 'Requirements Engineering', 'Software Design',
       'Software Testing', 'Software Maintenance',
       'Software Configuration Management', 'Software Metrics',
       'Finite Automata', 'Regular Languages', 'Context-Free Grammars',
       'Pumping Lemma', 'Decidability', 'Computability',
       'Introduction to Neural Networks', 'Convolutional Neural Networks',
       'Recurrent Neural Networks', 'Generative Models',
       'Transfer Learning', 'Deep Reinforcement Learning',
       'Applications of Deep Learning', 'Supervised Learning',
       'Unsupervised Learning', 'Neural Networks',
       'Reinforcement Learning', 'Introduction to NLP', 'Language Models',
       'Tokenization', 'Part-of-Speech Tagging',
       'Named Entity Recognition', 'Text Classification',
       'Sentiment Analysis', 'Language Generation', 'Machine Translation',
       'Topic Modeling', 'Information Retrieval', 'Text Summarization',
       'Speech Recognition', 'Machine Learning',
       'Natural Language Processing', 'Computer Vision',
       'Deterministic Finite Automata (DFA)',
       'Nondeterministic Finite Automata (NFA)', 'Regular Expressions',
       'Pushdown Automata (PDA)', 'Context-Free Languages',
       'Turing Machines', 'Complexity Theory', 'Lexical Analysis',
       'Syntax Analysis', 'Intermediate Code Generation',
       'Code Optimization', 'Code Generation', 'Cryptography',
       'Network Security', 'Web Security', 'Application Security',
       'Cloud Security', 'Threats and Attacks', 'Set Theory',
       'Combinatorics', 'Graph Theory', 'Number Theory', 'Logic',
       'Differentiation', 'Integration', 'Vector Calculus', 'Vectors',
       'Matrices', 'Eigenvalues and Eigenvectors',
       'Linear Transformations', 'Probability Theory',
       'Statistical Inference', 'Regression Analysis', 'Data Analysis',
       'Stochastic Processes', 'Time Series Analysis',
       'Classical Mechanics', 'Electromagnetism', 'Thermodynamics',
       'Modern Physics', 'Materials Science',
       'Atomic Structure and Chemical Bonding', 'Chemical Thermodynamics',
       'Chemical Kinetics', 'Chemical Equilibrium', 'Electrochemistry',
       'Organic Chemistry', 'Polymers', 'Surface Chemistry',
       'Environmental Chemistry', 'Statics', 'Dynamics', 'Vibrations',
       'Reinforced Concrete Design', 'Geotechnical Engineering',
       'Transportation Engineering', 'Environmental Engineering',
       'Construction Engineering and Management', 'Structural Dynamics',
       'Water Resources Engineering', 'Steel Structures',
       'Construction Materials', "Ohm's Law", 'Resistors', 'Capacitors',
       'Diodes', 'Transistors', 'Op-Amps', 'Logic Gates', 'PCB Design',
       'Analog and Digital Signals']

# Set the hyppothesis template
hypothesis_template = "The topic of this review is {}."

# Prediction results
single_topic_prediction = classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)

# Save the output as a dataframe
# single_topic_prediction = pd.DataFrame(single_topic_prediction)



Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f237a8cb250> was reported to be 297 (when accessing len(dataloader)), but 48585 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f237a8cb250> was reported to be 297 (when accessing len(dataloader)), but 48586 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f237a8cb250> was reported to be 297 (when accessing len(dataloader)), but 48587 samples have been fetched. 
  warnings.warn(warn_msg)
/

In [49]:
single_topic_prediction.head()

AttributeError: ignored

In [ ]:
# The column for the predicted topic
print(single_topic_prediction['labels'].apply(lambda x: x[0:5]))

# The column for the score of predi ted topic
single_topic_prediction['scores'].apply(lambda x: x[0:5])



In [ ]:
text='i want to learn about big O notation'
single_topic_prediction.predict

In [52]:
classifier.save_pretrained("/content/gdrive/academic_model")
classifier.tokenizer.save_pretrained("/content/gdrive/academic_model")

OSError: ignored

In [51]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive
